In [1]:
import pandas as pd
import numpy as np

## Viejo

In [67]:
df_original = pd.read_parquet(f'Pairs_SP500_FPT/spx_impl_vols.parquet')
df_original = df_original.dropna()
df_original['Date'] = pd.to_datetime(df_original['Date'],format="%Y-%m-%d")
df_original['Date'] = df_original['Date'].values.astype('datetime64[D]')
# df_original[~pd.isna(df_original['p_opt_atmiv_mkt'])].head()

## Nuevo

In [42]:
df_nuevo = pd.read_csv('20240529/daily_prices.csv', low_memory=False, index_col=0)
df_nuevo = df_nuevo.rename(columns={
    'symbol':'Id',
    'date':'Date',
    'price':'p_price',
    'implied_volatility':'p_opt_atmiv_mkt',
})
df_nuevo = df_nuevo[['Id','Date','p_price','p_opt_atmiv_mkt']]
df_nuevo = df_nuevo.dropna()
df_nuevo['Date'] = pd.to_datetime(df_nuevo['Date'],format="%Y-%m-%d")
df_nuevo['Date'] = df_nuevo['Date'].values.astype('datetime64[D]')

## Combinar

In [44]:
df = df_original.merge(df_nuevo,how='outer',on=['Id','Date'])
df.loc[(pd.isna(df['p_price_x']))&(~pd.isna(df['p_price_y'])),'p_price_x'] = df.loc[(pd.isna(df['p_price_x']))&(~pd.isna(df['p_price_y'])),'p_price_y']
df.loc[(pd.isna(df['p_opt_atmiv_mkt_x']))&(~pd.isna(df['p_opt_atmiv_mkt_y'])),'p_opt_atmiv_mkt_x'] = df.loc[(pd.isna(df['p_opt_atmiv_mkt_x']))&(~pd.isna(df['p_opt_atmiv_mkt_y'])),'p_opt_atmiv_mkt_y']
df = df.rename(columns={
    'p_price_x': 'p_price',
    'p_opt_atmiv_mkt_x': 'p_opt_atmiv_mkt'
})
df = df[['Id','Date','p_price','p_opt_atmiv_mkt']]
df = df.dropna()
df

,Id,Date,ff_isin,p_price_x,p_opt_atmiv_mkt_x,company,p_price_y,p_opt_atmiv_mkt_y
0,CTAS-US,2005-09-01,US1729081059,40.71,26.95001,CTAS-US,40.71,32.094870
1,CTAS-US,2005-09-02,US1729081059,40.25,21.69436,CTAS-US,40.25,23.876220
2,CTAS-US,2005-09-06,US1729081059,40.75,28.44466,CTAS-US,40.75,33.361030
3,CTAS-US,2005-09-07,US1729081059,41.20,29.96635,CTAS-US,41.20,37.774220
4,CTAS-US,2005-09-08,US1729081059,40.32,30.57159,CTAS-US,40.32,34.618400
...,...,...,...,...,...,...,...,...
2123074,SOLV-US,2024-05-21,NaN,NaN,NaN,NaN,61.22,36.488265
2123075,SOLV-US,2024-05-22,NaN,NaN,NaN,NaN,61.89,36.528057
2123076,SOLV-US,2024-05-23,NaN,NaN,NaN,NaN,59.93,36.383257
2123077,SOLV-US,2024-05-24,NaN,NaN,NaN,NaN,60.45,36.450295


In [58]:
df = df.loc[~df['Date'].isin(['2006-04-21','2006-06-14']),:]
df.to_parquet('Pairs_SP500_FPT/spx_impl_vols_new.parquet')

## Otros

In [12]:
nuevo = df_nuevo.groupby('Id').aggregate(func={'Date':['min','max']})
nuevo.columns = ['_'.join(col).strip() for col in nuevo.columns.values]
nuevo = nuevo.reset_index()
nuevo.head()

,Id,Date_min,Date_max
0,A-US,2005-09-01,2024-05-28
1,AAL-US,2013-12-10,2024-05-28
2,AAPL-US,2005-09-01,2024-05-28
3,ABBV-US,2013-01-02,2024-05-28
4,ABNB-US,2020-12-17,2024-05-28


In [13]:
min(nuevo['Date_min'])

'2005-09-01'

In [14]:
df_nuevo = pd.read_csv('20240529/daily_prices.csv', low_memory=False, index_col=0)
df_nuevo = df_nuevo.rename(columns={
    'symbol':'Id',
    'date':'Date',
    'price':'p_price',
    'implied_volatility':'p_opt_atmiv_mkt',
})
df_nuevo = df_nuevo[['Id','Date','p_price','p_opt_atmiv_mkt']]
df_nuevo['company'] = df_nuevo['Id']
df_nuevo = df_nuevo.reset_index(drop=True)
df_nuevo.describe()

,p_price,p_opt_atmiv_mkt
count,2.741667e+06,2.025213e+06
mean,8.028506e+01,3.359849e+01
std,1.710245e+02,1.206334e+02
min,3.052083e-02,9.999928e-03
25%,2.453000e+01,2.372048e+01
50%,4.500000e+01,2.884401e+01
75%,8.355000e+01,3.653161e+01
max,8.099960e+03,3.028871e+04


In [15]:
df_nuevo.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3088420 entries, 0 to 3088419
Data columns (total 5 columns):
 #   Column           Dtype  
---  ------           -----  
 0   Id               object 
 1   Date             object 
 2   p_price          float64
 3   p_opt_atmiv_mkt  float64
 4   company          object 
dtypes: float64(2), object(3)
memory usage: 117.8+ MB


In [16]:
df_nuevo.head()

,Id,Date,p_price,p_opt_atmiv_mkt,company
0,CTAS-US,1999-12-31,35.416668,NaN,CTAS-US
1,CTAS-US,2000-01-03,34.833332,NaN,CTAS-US
2,CTAS-US,2000-01-04,33.416668,NaN,CTAS-US
3,CTAS-US,2000-01-05,33.791668,NaN,CTAS-US
4,CTAS-US,2000-01-06,33.979168,NaN,CTAS-US


## Company data

- MKTval
- BTM: Book val/market cap
- PER: Market cap/Net Earnings
- EVEBIT: Enterprise value (Mkt cap + Net debt) / EBIT
- EVEBITDA: Enterprise value (Mkt cap + Net debt) / EBITDA
- GPA: Gross Profits/Assets
- ROC: Return on Capital, Net income/EV(Debt + Mkt cap)
- ROCD: Same but with Intangible assets on denominator
- MOM: 
- Global disclosure: 
- Environmental disclosure: 
- Social disclosure: 
- Governance disclosure: 
